In [17]:
def input(
        day: int,
        sample: bool=True,
        listify: bool=False):
    if sample:
        filename='sample'
    else:
        filename='input'
    with open(f'data/{day}/{filename}.txt') as file:
        data = file.read()

    if listify:
        data = data.splitlines()

    return data

# Day 1

In [92]:
inp = input(1, listify=True, sample=False)

total = 0 
for line in inp:
    nums = []
    for char in line:
        if char in '1234567890':
            nums.append(char)
    total += int(nums[0]+nums[-1])

print(total)

swap = {
    'one':1, 'two':2, 'three':3,
    'four':4,'five':5, 'six':6,
    'seven':7, 'eight':8, 'nine':9
}

total = 0 
for line in inp:
    for key in swap.keys():
        line = line.replace(key, key+str(swap[key])+key)
    nums = []
    for char in line:
        if char in '1234567890':
            nums.append(char)
    total += int(nums[0]+nums[-1])

print(total)

53921
54676
